In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Carregando os dados
data = pd.read_csv('features_and_decisions.csv')

# Preparando os dados
features = data.drop(columns=['screenshot_title', 'decision_activity', 'decision_hearts', 'decision_light', 'decision_mob'])
label_encoders = {}

for col in features.select_dtypes(include='object').columns:
    le = LabelEncoder()
    features[col] = le.fit_transform(features[col])
    label_encoders[col] = le

target = data['decision_mob']
target_encoder = LabelEncoder()
target = target_encoder.fit_transform(target)

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

# Configurar pandas para exibir todas as linhas e colunas
#pd.set_option('display.max_rows', None)  # Exibir todas as linhas
#pd.set_option('display.max_columns', None)  # Exibir todas as colunas
#pd.set_option('display.width', None)  # Ajustar a largura
#pd.set_option('display.max_colwidth', None)  # Exibir conteúdo completo das colunas

# Resetar as opções de exibição para o padrão
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_colwidth')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.width')


features



,activity,hearts,light_lvl,in_hand_item,target_mob
0,0,11,0,2,2
1,0,3,0,2,2
2,0,3,0,2,2
3,0,20,0,2,2
4,0,15,0,2,3
...,...,...,...,...,...
6084,4,4,0,5,2
6085,4,4,0,5,2
6086,4,4,0,5,2
6087,4,4,0,5,2


In [2]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Treinando o modelo SVM com kernel RBF
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = svm_model.predict(X_test)

# Avaliação
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.4f}')
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))


Acurácia do modelo: 0.8949
                     precision    recall  f1-score   support

            go_back       0.98      1.00      0.99        40
no_decision_for_mob       0.92      0.94      0.93       918
           take_bow       0.64      0.61      0.63       114
         take_sword       0.88      0.79      0.83       146

           accuracy                           0.89      1218
          macro avg       0.85      0.84      0.85      1218
       weighted avg       0.89      0.89      0.89      1218



In [3]:
import pygad
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Função de fitness para otimização dos parâmetros com k-fold
def fitness_function(ga_instance, solution, solution_idx):
    C, gamma = solution
    model = SVC(kernel='rbf', C=C, gamma=gamma, random_state=42)
    
    # Validando a performance do modelo usando validação cruzada k-fold
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')  # cv=5 para k-fold
    return np.mean(scores),  # Retorna uma tupla (PyGAD exige isso)



In [4]:
# Definindo a estrutura do algoritmo genético
def run_ga():
    sol_per_pop = 10  # Número de soluções na população
    num_genes = 2  # Número de parâmetros: C e gamma

    # Definir o intervalo para C e gamma
    gene_space = [{'low': 0.1, 'high': 1000},  # C
                  {'low': 0.001, 'high': 1}]  # gamma

    # Inicializando o PyGAD
    ga_instance = pygad.GA(num_generations=10,
                           num_parents_mating=5,  # número de pais para a reprodução
                           sol_per_pop=sol_per_pop,  # soluções por população
                           num_genes=num_genes,  # número de parâmetros
                           gene_space=gene_space,  # intervalo de valores para cada gene
                           fitness_func=fitness_function,  # função de fitness
                           crossover_type="uniform",  # tipo de crossover
                           crossover_probability=0.8,  # probabilidade de crossover
                           mutation_type="random",  # tipo de mutação
                           mutation_probability=0.2,  # probabilidade de mutação
                           parent_selection_type="tournament",  # seleção dos pais
                           keep_parents=1)  # número de pais a serem mantidos

    # Executando a otimização
    ga_instance.run()

    # Acompanhando o progresso por geração
    for generation in range(ga_instance.num_generations):
        best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
        print(f"Geração {generation + 1} - Melhor fitness: {best_solution_fitness}")
        print(f"Melhor C: {best_solution[0]}, Melhor Gamma: {best_solution[1]}")

    # Obtendo os melhores parâmetros
    best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
    best_C, best_gamma = best_solution
    print(f'Melhor C final: {best_C}, Melhor Gamma final: {best_gamma}')
    return best_C, best_gamma

In [5]:
best_C, best_gamma = run_ga()


c:\Users\emanu\my-project\.venv\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Geração 1 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 2 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 3 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 4 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 5 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 6 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 7 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 8 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 9 - Melhor fitness: [0.98378118]
Melhor C: 64.38479532414553, Melhor Gamma: 0.05432799977559431
Geração 10 - Melhor fitness: [0.98378118]
Melhor C: 64.384795324

In [6]:
# Treinar o modelo SVM com os melhores parâmetros
svm_optimized = SVC(kernel='rbf', C=best_C, gamma=best_gamma, random_state=42)
svm_optimized.fit(X_train, y_train)

# Avaliação no conjunto de teste
y_pred_optimized = svm_optimized.predict(X_test)
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)

print(f'Acurácia otimizada: {accuracy_optimized:.4f}')
print(classification_report(y_test, y_pred_optimized, target_names=target_encoder.classes_))


Acurácia otimizada: 0.9860
                     precision    recall  f1-score   support

            go_back       1.00      1.00      1.00        40
no_decision_for_mob       1.00      0.98      0.99       918
           take_bow       0.99      1.00      1.00       114
         take_sword       0.91      0.99      0.94       146

           accuracy                           0.99      1218
          macro avg       0.97      0.99      0.98      1218
       weighted avg       0.99      0.99      0.99      1218

